In [1]:
import gym
import os
import csv

import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim

from buffer import TraceBuf
from common import epsilon_at, downsample, to_grayscale, checkpoint_dir, checkpoint_exists
from common_tf import checkpoint, load_checkpoint

In [2]:
class Qnetwork():
    def __init__(self, h_size, a_size, rnn_cell, scopeName):
        self.h_size, self.a_size = h_size, a_size
        self.scalarInput = tf.placeholder(shape=[None, 7056], dtype=tf.float32)
        self.batch_size = tf.placeholder(dtype=tf.int32, shape=[])
        self.trainLength = tf.placeholder(dtype=tf.int32, shape=[])
        
        self.frameShape = tf.constant((84, 84, 1), dtype=tf.int32)
#         self.frames = tf.reshape(self.scalarInput, tf.concat(([self.batch_size*self.trainLength], self.frameShape), 0))
        self.frames = tf.reshape(self.scalarInput, [-1, 84, 84, 1])
        self.conv1 = slim.convolution2d(
            inputs=self.frames, num_outputs=32,
            kernel_size=(8, 8), stride=(4, 4), padding='VALID',
            biases_initializer=None, scope=scopeName+'_conv1'
        )
        self.conv2 = slim.convolution2d(
            inputs=self.conv1, num_outputs = 64,
            kernel_size=(4, 4), stride=(2, 2), padding='VALID',
            biases_initializer=None, scope=scopeName+'_conv2'
        )
        self.conv3 = slim.convolution2d(
            inputs=self.conv2, num_outputs = 64,
            kernel_size=(3, 3), stride=(1, 1), padding='VALID',
            biases_initializer=None, scope=scopeName+'_conv3'
        )
        self.conv4 = slim.convolution2d(
            inputs=self.conv3, num_outputs = h_size,
            kernel_size=(7, 7), stride=(1, 1), padding='VALID',
            biases_initializer=None, scope=scopeName+'_conv4'
        )
        
        self.convFlat = tf.reshape(
            slim.flatten(self.conv4), [self.batch_size, self.trainLength, h_size])
        
        self.state_init = rnn_cell.zero_state(self.batch_size, tf.float32)
        self.rnn, self.rnn_state = tf.nn.dynamic_rnn(
            inputs=self.convFlat, cell=rnn_cell, dtype=tf.float32,
            initial_state=self.state_init, scope=scopeName+'_rnn'
        )
        self.rnn = tf.reshape(self.rnn, shape=[-1, h_size])
        
        self.streamA, self.streamV = tf.split(self.rnn, 2, axis=1)
        
        self.AW = tf.Variable(tf.random_normal([h_size//2, a_size]))
        self.VW = tf.Variable(tf.random_normal([h_size//2, 1]))
        self.A = tf.matmul(self.streamA, self.AW)
        self.V = tf.matmul(self.streamV, self.VW)
        
        self.salience = tf.gradients(self.A, self.scalarInput)
        
        self.Qout = self.V + \
            (self.A - tf.reduce_mean(self.A, axis=1, keepdims=True))
        self.predict = tf.argmax(self.Qout, 1)
        self.action = self.predict[-1]
        
        self.targetQ = tf.placeholder(shape=[None], dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None], dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions, a_size, dtype=tf.float32)
        
        self.Q = tf.reduce_sum(self.Qout * self.actions_onehot,
                               reduction_indices=1)
        
        # only train on first half of trace per Lample & Chatlot 2016
        self.mask = tf.concat((tf.zeros((self.batch_size, self.trainLength//2)),
                               tf.ones((self.batch_size, self.trainLength//2))), 1)
        self.mask = tf.reshape(self.mask, [-1])
        
        self.loss = tf.losses.huber_loss(self.Q * self.mask, self.targetQ * self.mask)
        tf.summary.histogram('loss', self.loss)
        
        self.trainer = tf.train.RMSPropOptimizer(0.00025, momentum=0.95, epsilon=0.01)
        self.updateModel = self.trainer.minimize(self.loss)
        
        

In [3]:
def preprocess(s):
    return downsample(to_grayscale(s[8:-12]), (84,84)).reshape((7056,))

def reset(env, buf, noop_max=30):
    if buf.trans_cache: buf._flush()
    S = [preprocess(env.reset())]
    life = None
    for _ in range(np.random.randint(noop_max)):
        s, r, t, l = env.step(0)
        s, life = preprocess(s), l['ale.lives']
        if t: return reset(env, buf, noop_max)
        buf.append_trans((S[-1],0,r,s,t))
        S.append(s)
    return S, life

In [4]:

def getTargetUpdateOps(tfVars):
    # tfVars consists of all trainble TF Variables
    # where the first half is from the main network
    #  and the second half is from the target network
    # RETURNS: list of operations which when run, 
    #          updates the target network with main network's values
    return [vt.assign(vm.value()) 
            for i, (vm, vt) in enumerate(zip(tfVars[:len(tfVars)//2],
                                             tfVars[len(tfVars)//2:]))]

In [5]:
def updateTarget(op_holder, sess):
    for op in op_holder:
        sess.run(op)
    total_vars = len(tf.trainable_variables())
    a = tf.trainable_variables()[0].eval(session=sess)
    b = tf.trainable_variables()[total_vars//2].eval(session=sess)
    if a.all() == b.all():
        print("Target Set Success")
    else:
        print("Target Set Failed")

In [6]:

def train(trace_length, h_size=512, update_freq=4, ckpt_freq=10000, batch_size=32, env_name='SpaceInvaders', total_iteration=5e7, pretrain_steps=50000):
    global exp_buf
    env_name += 'NoFrameskip-v4'
    identity = 'stack={},env={},mod={}'.format(trace_length, env_name, 'drqn')
    
    
    env = gym.make(env_name)
    a_size = env.action_space.n
    
    tf.reset_default_graph()
    cell  = tf.nn.rnn_cell.LSTMCell(num_units=h_size)
    cellT = tf.nn.rnn_cell.LSTMCell(num_units=h_size)
    mainQN   = Qnetwork(h_size, a_size, cell, 'main')
    targetQN = Qnetwork(h_size, a_size, cellT, 'target')
    init = tf.global_variables_initializer()
    updateOps = getTargetUpdateOps(tf.trainable_variables())
    saver = tf.train.Saver(max_to_keep=5)

    sess = tf.Session()
    summary_writer = tf.summary.FileWriter('./log/' + identity, sess.graph)
    
    
    if checkpoint_exists(identity):
        (exp_buf, env, last_iteration, is_done, 
         prev_life_count, prev_action,
         prev_action_taken, state) = load_checkpoint(sess, saver, identity)
    else:
        exp_buf = TraceBuf(trace_length, size=50000)
        last_iteration = 1 - pretrain_steps
        is_done = True
        prev_life_count = None
        prev_action, prev_action_taken = 0, 0
        state = None
    
    sess.run(init)
    
    updateTarget(updateOps, sess)
    
#     print((exp_buf, env, last_iteration, is_done, 
#          prev_life_count, prev_action,
#          prev_action_taken, state))
#     print(last_iteration)
    
    for i in range(last_iteration, int(total_iteration)):
        if is_done:
            exp_buf._flush()
            S, prev_life_count = reset(env, exp_buf)
            state, action = sess.run((mainQN.rnn_state, mainQN.action), feed_dict={
                mainQN.scalarInput: np.vstack(np.array(S)/255.0),
                mainQN.trainLength: len(S),
                mainQN.state_init: (np.zeros((1, h_size)),) * 2,
                mainQN.batch_size: 1
            })
            
        
        S = [S[-1]]
        for _ in range(4):
            s, r, is_done, info = env.step(action)
            s, life_count = preprocess(s), info['ale.lives']
            exp_buf.append_trans((
                S[-1], action, np.sign(r), s,
                (prev_life_count and life_count < prev_life_count or is_done)
            ))
            S.append(s)
            prev_life_count = life_count
            
        feed = {
            mainQN.scalarInput: np.vstack(np.array(S[1:])/255.0),
            mainQN.trainLength: 4,
            mainQN.state_init: state,
            mainQN.batch_size: 1
        }
        if np.random.random() < epsilon_at(i):
            action = env.action_space.sample()
            state = sess.run(mainQN.rnn_state, feed_dict=feed)
        else:
            action, state = sess.run((mainQN.action, mainQN.rnn_state),\
                                        feed_dict=feed)
        
        
        if not i: start_time = time.time()
            
        if i <= 0: continue
            
        if not i % ckpt_freq:
            checkpoint(sess, saver, identity,
                       exp_buf, env, last_iteration, is_done, 
                       prev_life_count, prev_action,
                       prev_action_taken, state)

        if not i % (update_freq * 1000):
            updateTarget(updateOps, sess)
#             print(evaluate(sess, mainQN, env_name))
            print(i)
            cur_time = time.time()
            print('took', cur_time-start_time, 'second to do', str(update_freq*1000), 'iterations')
            start_time = cur_time

        if not i % update_freq:
            # train batch
            summaryOps = tf.summary.merge_all()
            
            state_train = (np.zeros((batch_size, h_size)),) * 2

            trainBatch = exp_buf.sample_traces(batch_size)
            

            Q1 = sess.run(mainQN.predict, feed_dict={
                mainQN.scalarInput: np.vstack(trainBatch[:,3]/255.0),
                mainQN.trainLength: trace_length,
                mainQN.state_init: state_train,
                mainQN.batch_size: batch_size
            })
            Q2 = sess.run(targetQN.Qout, feed_dict={
                targetQN.scalarInput: np.vstack(trainBatch[:,3]/255.0),
                targetQN.trainLength: trace_length,
                targetQN.state_init: state_train,
                targetQN.batch_size: batch_size
            })
            end_multiplier = - (trainBatch[:,4] - 1)
            doubleQ = Q2[range(batch_size * trace_length), Q1]
            targetQ = trainBatch[:,2] + (0.99 * doubleQ * end_multiplier)

            print(trainBatch[:,0].shape)
            _, loss, summary = sess.run((mainQN.updateModel, mainQN.loss, summaryOps), feed_dict={
                mainQN.scalarInput:np.vstack(trainBatch[:,0]/255.0),
                mainQN.targetQ: targetQ,
                mainQN.actions: trainBatch[:,1],
                mainQN.trainLength: trace_length,
                mainQN.state_init: state_train,
                mainQN.batch_size: batch_size
            })

            print(summary)
            summary_writer.add_summary(summary, i)
            
        
        
    sess.close()
    checkpoint(sess, saver, identity)
    

In [7]:
import time
def evaluate(sess, mainQN, env_name, action_repeat=6, scenario_count=3, is_render=False):
    start_time = time.time()
    env = gym.make(env_name)
    # step 6 frame with same action, and use trace size = 6
    def get_action(S):
        return sess.run(mainQN.action, feed_dict={
            mainQN.scalarInput: np.vstack(np.array(S)/255.0),
            mainQN.trainLength: len(S),
            mainQN.state_init: (np.zeros((1, mainQN.h_size)),) * 2,
            mainQN.batch_size: 1
        })
    
    def run_scenario():
        S, R, t = [preprocess(env.reset())], 0, 0
        noop = np.random.randint(30)
        for i in range(noop):
            frame, r, t, _ = env.step(0)
            S += (preprocess(frame),)
            R += r
        action = get_action(S)
        while not t:
            S.clear()
            for i in range(action_repeat):
#                 print(action)
                frame, r, t, _ = env.step(action)
                R += r
                S += (preprocess(frame),)
                if is_render: env.render()
            action = get_action(S)
        return R

    res = np.array([run_scenario() for _ in range(scenario_count)])
    print(time.time()-start_time, 'seconds to evaluate')
    return np.mean(res), np.std(res)
                
    

In [8]:
train(10, pretrain_steps=8000)

Target Set Success
(320,)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_5' with dtype float and shape [?,7056]
	 [[Node: Placeholder_5 = Placeholder[dtype=DT_FLOAT, shape=[?,7056], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: huber_loss/value/_119 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_275_huber_loss/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_5', defined at:
  File "/home/diyiw/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/diyiw/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/diyiw/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/diyiw/an

In [9]:
exp_buf

In [11]:
traces = exp_buf.sample_traces(32)

In [ ]:
np.vstack(traces[:,0]/255.0).shape

In [17]:
traces[:,1].shape

(320,)

In [ ]:
# env = gym.make('SpaceInvadersNoFrameskip-v4')
# s = env.reset()
# p = preprocess(s)
# print(p.shape)